<a href="https://colab.research.google.com/github/saivaruntejamudumba/INFO-5731/blob/master/Mudumba_SelfStudy7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from fastai.text import *

In [2]:
path = untar_data(URLs.IMDB_SAMPLE)
path.ls()

[PosixPath('/root/.fastai/data/imdb_sample/texts.csv')]

In [3]:
df = pd.read_csv(path/'texts.csv')
df.head()

,label,text,is_valid
0,negative,Un-bleeping-believable! Meg Ryan doesn't even ...,False
1,positive,This is a extremely well-made film. The acting...,False
2,negative,Every once in a long while a movie will come a...,False
3,positive,Name just says it all. I watched this movie wi...,False
4,negative,This movie succeeds at being one of the most u...,False


In [4]:
df['text'][1]

'This is a extremely well-made film. The acting, script and camera-work are all first-rate. The music is good, too, though it is mostly early in the film, when things are still relatively cheery. There are no really superstars in the cast, though several faces will be familiar. The entire cast does an excellent job with the script.<br /><br />But it is hard to watch, because there is no good end to a situation like the one presented. It is now fashionable to blame the British for setting Hindus and Muslims against each other, and then cruelly separating them into two countries. There is some merit in this view, but it\'s also true that no one forced Hindus and Muslims in the region to mistreat each other as they did around the time of partition. It seems more likely that the British simply saw the tensions between the religions and were clever enough to exploit them to their own ends.<br /><br />The result is that there is much cruelty and inhumanity in the situation and this is very u

In [0]:
data_lm = TextDataBunch.from_csv(path, 'texts.csv')

In [0]:
data_lm.save()

In [0]:
data = load_data(path)

In [8]:
data = TextClasDataBunch.from_csv(path, 'texts.csv')
data.show_batch()

text,target
"xxbos xxmaj raising xxmaj victor xxmaj vargas : a xxmaj review \n \n xxmaj you know , xxmaj raising xxmaj victor xxmaj vargas is like sticking your hands into a big , steaming bowl of xxunk . xxmaj it 's warm and gooey , but you 're not sure if it feels right . xxmaj try as i might , no matter how warm and gooey xxmaj raising xxmaj",negative
"xxbos xxup the xxup shop xxup around xxup the xxup corner is one of the xxunk and most feel - good romantic comedies ever made . xxmaj there 's just no getting around that , and it 's hard to actually put one 's feeling for this film into words . xxmaj it 's not one of those films that tries too hard , nor does it come up with",positive
"xxbos xxmaj now that xxmaj che(2008 ) has finished its relatively short xxmaj australian cinema run ( extremely limited xxunk screen in xxmaj sydney , after xxunk ) , i can xxunk join both xxunk of "" xxmaj at xxmaj the xxmaj movies "" in taking xxmaj steven xxmaj soderbergh to task . \n \n xxmaj it 's usually satisfying to watch a film director change his style /",negative
"xxbos xxmaj this film sat on my xxmaj tivo for weeks before i watched it . i xxunk a self - indulgent xxunk flick about relationships gone bad . i was wrong ; this was an xxunk xxunk into the screwed - up xxunk of xxmaj new xxmaj yorkers . \n \n xxmaj the format is the same as xxmaj max xxmaj xxunk ' "" xxmaj la xxmaj ronde",positive
"xxbos xxmaj many xxunk that this is n't just a classic due to the fact that it 's the first xxup 3d game , or even the first xxunk - up . xxmaj it 's also one of the first stealth games , one of the xxunk definitely the first ) truly claustrophobic games , and just a pretty well - rounded gaming experience in general . xxmaj with graphics",positive


In [9]:
data.vocab.itos[:10]

['xxunk',
 'xxpad',
 'xxbos',
 'xxfld',
 'xxmaj',
 'xxup',
 'xxrep',
 'xxwrep',
 'the',
 ',']

In [10]:
data.train_ds[0][0]

Text xxbos a vampire 's 's henchman wants to call her after falling in love with a five - dollar hooker in this extremely low - budget horror - comedy . i ca n't explain all the positive comments on this movie . i 'll chalk it up to mass hallucination , but it 's disconcerting none the less . xxmaj the one redeeming factor ( and this is me being extremely generous here ) might be the xxmaj grandfather who 's the only semi - likable character in this whole mess . xxmaj do n't waste your money , or time . xxmaj in fact here 's a word of advice , xxmaj if xxmaj troma puts it out on xxup dvd , but does xxup not make it themselves , in all likelihood it 's crap . 
 
  xxmaj troma xxup dvd xxmaj extras : xxmaj commentary with xxmaj omar and xxmaj kirk ; second commentary with cast and crew deleted scenes ; xxunk ; troma xxunk ; radiation march ; xxmaj clip from " xxmaj terror xxmaj xxunk " ; xxmaj theatrical trailer ; xxmaj trailers for " the xxmaj xxunk xxmaj girls " , " xxmaj teenage xxmaj

In [11]:
data.train_ds[0][0].data[:10]

array([   2,   12, 1191,   22,   22, 6129,  569,   14,  801,   58])

In [0]:
data = (TextList.from_csv(path, 'texts.csv', cols='text')
                .split_from_df(col=2)
                .label_from_df(cols=0)
                .databunch())

In [0]:
bs=48

In [14]:
path = untar_data(URLs.IMDB)
path.ls()

[PosixPath('/root/.fastai/data/imdb/test'),
 PosixPath('/root/.fastai/data/imdb/tmp_lm'),
 PosixPath('/root/.fastai/data/imdb/tmp_clas'),
 PosixPath('/root/.fastai/data/imdb/README'),
 PosixPath('/root/.fastai/data/imdb/imdb.vocab'),
 PosixPath('/root/.fastai/data/imdb/train'),
 PosixPath('/root/.fastai/data/imdb/unsup')]

In [15]:
(path/'train').ls()

[PosixPath('/root/.fastai/data/imdb/train/neg'),
 PosixPath('/root/.fastai/data/imdb/train/pos'),
 PosixPath('/root/.fastai/data/imdb/train/unsupBow.feat'),
 PosixPath('/root/.fastai/data/imdb/train/labeledBow.feat')]

In [0]:
data_lm = (TextList.from_folder(path)
           #Inputs: all the text files in path
            .filter_by_folder(include=['train', 'test', 'unsup']) 
           #We may have other temp folders that contain text files so we only keep what's in train and test
            .split_by_rand_pct(0.1)
           #We randomly split and keep 10% (10,000 reviews) for validation
            .label_for_lm()           
           #We want to do a language model so we label accordingly
            .databunch(bs=bs))
data_lm.save('data_lm.pkl')

In [0]:
data_lm = load_data(path, 'data_lm.pkl', bs=bs)

In [18]:
data_lm.show_batch()

idx,text
0,"( i 'll leave out the videography , special effects , and artistic ability in this review . ) xxmaj what is most disappointing is the historical inaccuracy of this movie and how it is so far from the historical accounts from xxmaj biblical texts . xxmaj one of the overarching principles from the xxmaj bible is that * xxmaj god * led xxmaj his people out of xxmaj egypt"
1,", i would n't even be writing this review . xxmaj the ending of this movie is so corny , you wo n't believe . \n \n xxmaj tim xxmaj matheson is a xxmaj made - for - xxup tv - xxmaj movie king . he just looks like such a xxunk mummy in this movie . xxmaj he 's a little wooden and stiff . xxmaj the dazzling"
2,"is n't entirely xxup cgi rendered . xxmaj so even though it does n't look good , at least there are a few legit scenes of characters inanely grappling with rubbery tentacles in pretty amusing fashion . i also chuckled at the disparities between different representations of the octopus , the cgi shots of the creature as a whole vary over the course of the film , they are also"
3,"carnage ensues . xxmaj that 's pretty much it as far as plot development goes . xxmaj it worked nicely in the first part because of the ghoulish make - up xxup fx , fast pace and unpredictability . xxmaj the sequel , however , does n't cut it . xxmaj the first problem seems to be that there are way too many characters who we do n't really care"
4,"when xxmaj finney , who has broken his leg , is playfully jumped on by all the other boys during a ball game . xxmaj unless they were just a bunch of nincompoops , they would know they could not possibly throw their bodies against him . xxmaj obviously this bone - shattering moment was lost on both the director and the producer . xxbos xxmaj is it really possible"


In [0]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.3)

In [0]:
learn.lr_find()

epoch,train_loss,valid_loss,accuracy,time


In [0]:
learn.recorder.plot(skip_end=15)

In [0]:
learn.fit_one_cycle(1, 1e-2, moms=(0.8,0.7))

In [0]:
learn.save('first')

In [0]:
learn.load('first')

In [0]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(1e-2/(2.6**4),1e-2), moms=(0.8,0.7))

In [0]:
learn.save('second')

In [0]:
learn.load('second')

In [0]:
learn.freeze_to(-3)
learn.fit_one_cycle(1, slice(5e-3/(2.6**4),5e-3), moms=(0.8,0.7))

In [0]:
learn.save('third')

In [0]:
learn.load('third')

In [0]:
learn.unfreeze()
learn.fit_one_cycle(2, slice(1e-3/(2.6**4),1e-3), moms=(0.8,0.7))

In [0]:
learn.predict("I really loved that movie, it was awesome!")